## スクレイピング

In [1]:
# 時間を制御する time モジュールをインポート
from time import sleep

# HTTP通信ライブラリの requests モジュールをインポート
from requests import get

# 日付を扱うための datetime モジュールをインポート
from datetime import datetime as dt
from datetime import timedelta as td
from datetime import date

# フォルダ(ディレクトリ)を作成する makedirs モジュールをインポート
from os import makedirs
# ファイルの保存先を指定
SAVE_DIR = "results_lzh/"

# リクエスト間隔を指定(秒)　※サーバに負荷をかけないよう3秒以上を推奨
INTERVAL = 3

# URLの固定部分を指定
FIXED_URL = "http://www1.mbrace.or.jp/od2/B/"

# 開始合図
print("作業を開始します")

# ファイルを格納するフォルダを作成
makedirs(SAVE_DIR, exist_ok=True)

# 本日の日付を取得
today = date.today()

# datetime オブジェクトに変換 (時刻は 00:00:00 になります)
start_date = dt.combine(today, dt.min.time())
end_date = dt.combine(today, dt.min.time())

# 日付の差から期間を計算
days_num = (end_date - start_date).days + 1

# 日付リストを格納する変数
date_list = []

# 日付リストを生成
for i in range(days_num):
    # 開始日から日付を順に取得
    target_date = start_date + td(days=i)

    # 日付型を文字列に変換してリストに格納(YYYYMMDD)
    date_list.append(target_date.strftime("%Y%m%d"))

# URL生成とダウンロード
for date in date_list:

    # URL生成
    yyyymm = date[0:4] + date[4:6]
    yymmdd = date[2:4] + date[4:6] + date[6:8]

    variable_url = FIXED_URL + yyyymm + "/b" + yymmdd + ".lzh"
    file_name = "b" + yymmdd + ".lzh"

    # ダウンロード
    r = get(variable_url)

    # 成功した場合
    if r.status_code == 200:
        f = open(SAVE_DIR + file_name, 'wb')
        f.write(r.content)
        f.close()
        print(variable_url + " をダウンロードしました")

    # 失敗した場合
    else:
        print(variable_url + " のダウンロードに失敗しました")

    # 指定した間隔をあける
    sleep(INTERVAL)

# 終了合図
print("作業を終了しました")

作業を開始します
http://www1.mbrace.or.jp/od2/B/202503/b250322.lzh をダウンロードしました
作業を終了しました


In [ ]:
import re
# LZH形式のファイルを解凍するパッケージ lhafile をインポート
import lhafile
import os

# ダウンロードしたLZHファイルが保存されている場所を指定
LZH_FILE_DIR = "results_lzh"

# 解凍したファイルを保存する場所を指定
TXT_FILE_DIR = "B当日"

# 開始合図
print("作業を開始します")

# ファイルを格納するフォルダを作成
os.makedirs(TXT_FILE_DIR, exist_ok=True)

# LZHファイルのリストを取得
lzh_file_list = os.listdir(LZH_FILE_DIR)

# 正規表現パターンを定義（日付が含まれる部分にマッチするパターン）
pattern = r'[bk]\d{6}\.lzh'

# ファイルの数だけ処理を繰り返す
for lzh_file_name in lzh_file_list:

    # 正規表現パターンにマッチする場合にのみ実行
    if re.match(pattern, lzh_file_name):

        file = lhafile.Lhafile(os.path.join(LZH_FILE_DIR, lzh_file_name))

        # 解凍したファイルの名前を取得
        info = file.infolist()
        name = info[0].filename

        # 解凍したファイルの保存
        open(os.path.join(TXT_FILE_DIR, name), "wb").write(file.read(name))

        print(TXT_FILE_DIR + name + " を解凍しました")

# 終了合図
print("作業を終了しました")

作業を開始します
B当日B250322.TXT を解凍しました
作業を終了しました


In [3]:
import glob
import os
import re
from string import ascii_letters
from string import digits
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import japanize_matplotlib
import lightgbm as lgb
import pickle

In [4]:
files_B = glob.glob("B当日/*")

In [5]:
# Bデータを全て取り込み結合する
df_all_B = pd.DataFrame()

for file in files_B:

  #データ読み込み
  with open(file, encoding='shift-jis') as f:
    data1 = f.readlines()

  #レース日付読み込み
  date = os.path.basename(file)
  date = re.sub(r"\D", "", date)

  #余分なスペース、改行を消去
  data2 = [s.replace('\u3000', '').replace('\n', '') for s in data1]

  #全角文字を半角文字に変換
  han = ascii_letters + digits
  table = {c + 65248: c for c in map(ord, han)}
  data3 = [name.translate(table) for name in data2]

  #必要データ抽出
  data4 = [row for row in data3 if re.match('^[0-9]', row)]

  #行検索用
  pattern_place1 = '\d{2}[B][B]'
  pattern_race_num1 = '\d+[R]'
  pattern_racer1 = '^[1-6]\s\d{4}'
  pattern_place_re1 = re.compile(pattern_place1)
  pattern_race_num_re1 = re.compile(pattern_race_num1)
  pattern_racer_re1 = re.compile(pattern_racer1)


  #データ取得用
  pattern_place2 = '(\d{2})[B][B]'
  pattern_race_num2 = '(\d+)[R]'
  pattern_racer2 = '^([1-6])\s(\d{4})([^0-9]+)(\d{2})([^0-9]+)(\d{2})([AB]\d{1})\s(\d.\d{2})\s*(\d+.\d{2})\s*(\d+.\d{2})\s*(\d+.\d{2})\s*(\d+)\s*(\d+.\d{2})\s*(\d+)\s*(\d+.\d{2})'
  pattern_place_re2 = re.compile(pattern_place2)
  pattern_race_num_re2 = re.compile(pattern_race_num2)
  pattern_racer_re2 = re.compile(pattern_racer2)

  #必要データ取得
  values = []
  for row in data4:
    if re.match(pattern_place_re1, row):
      place = re.match(pattern_place_re2, row).groups()
      place_elm = place[0]

    elif re.match(pattern_race_num_re1, row):
      race_num = re.match(pattern_race_num_re2, row).groups()
      race_num_elm = race_num[0].zfill(2)

    elif re.match(pattern_racer_re1, row):
      value = re.match(pattern_racer_re2, row).groups()
      val_li = []
      for i in value:
        val_li.append(i)
      val_li.append(place_elm)
      val_li.append(race_num_elm)
      val_li.append(date)
      values.append(val_li)

  #データフレーム作成
  column = ['艇番', '選手登番', '選手名', '年齢', '支部', '体重', '級別', '全国勝率', '全国2連率', '当地勝率', '当地2連率', 'モーターNO', 'モーター2連率', 'ボートNO', 'ボート2連率', '開催地', 'レース番号', '日付']
  df_B = pd.DataFrame(values, columns=column)

  #開催地、レース番号、日付(すべてstr型)を元ににレースIDを追加
  df_B['レースID'] = df_B['日付'] + df_B['開催地'] + df_B['レース番号']

  df_all_B = pd.concat([df_all_B, df_B])

  df = df_all_B

# 以下はモデル作成時

In [6]:
#体重、年齢、艇番、レースIDをstr型からint型に変換
df[['体重', '年齢', '艇番','選手登番','モーターNO','ボートNO','開催地',]] = df[['体重', '年齢', '艇番','選手登番','モーターNO','ボートNO','開催地',]].astype(int)
df[['レースID']] = df[['レースID']].astype(np.int64)

# df = df.drop('選手名', axis=1)
df = df.drop('レース番号', axis=1)
df = df.drop('日付', axis=1)
df = df.drop('ボートNO', axis=1)

In [7]:
# データフレーム df の '級別' 列の値を変換
df['級別'] = df['級別'].replace({'A1': 1, 'A2': 2, 'B1': 3, 'B2': 4})

/tmp/ipykernel_2691/3512276922.py:2: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df['級別'] = df['級別'].replace({'A1': 1, 'A2': 2, 'B1': 3, 'B2': 4})


In [8]:
df['全国勝率'] = df['全国勝率'].astype("float64")
df['全国2連率'] = df['全国2連率'].astype("float64")
df['当地勝率'] = df['当地勝率'].astype("float64")
df['当地2連率'] = df['当地2連率'].astype("float64")
df['モーター2連率'] = df['モーター2連率'].astype("float64")
df['ボート2連率'] = df['ボート2連率'].astype("float64")

In [9]:
base_df = df

df = df.drop(['支部','選手名','レースID','選手登番','開催地',
             ], axis=1)

In [10]:
columns_list = df.columns.tolist()
print(columns_list)

['艇番', '年齢', '体重', '級別', '全国勝率', '全国2連率', '当地勝率', '当地2連率', 'モーターNO', 'モーター2連率', 'ボート2連率']


In [11]:
# 学習モデルmodel.pklの読み込み
with open('/home/jovyan/work/Learning_model/model.pkl', 'rb') as model_file:
	model = pickle.load(model_file)

# 予測を行う
# prediction_columns = ['艇番', '当地勝率','モーター2連率','選手のコース複勝率','選手のコース平均スタートタイミング']
predictions = model.predict(df)

# 予測結果を0または1に変換
binary_predictions = (predictions >= 0.5).astype(int)


In [12]:
# 予測確率を出力
df_result = pd.DataFrame({'1の確率':predictions})
df_result['レースID'] = base_df['レースID'].values  # 'レースID'列を追加
# 'レースID' カラムの各値の先頭に '20' を追加
#df_result['レースID'] = '20' + df_result['レースID'].astype(str)
df_result['艇番'] = base_df['艇番'].values  # 'レースID'列を追加
df_result['選手名'] = base_df['選手名'].values  # 'レースID'列を追加
df_result = df_result[['レースID', '1の確率', '艇番','選手名']]

# 'レースID'ごとに'1の確率'の値が大きい順にソートし、順位を付ける
df_result['予測順位'] = df_result.groupby('レースID')['1の確率'].rank(ascending=False)

In [13]:
df_result

,レースID,1の確率,艇番,選手名,予測順位
0,2503222401,0.361458,1,田中宏典,1.0
1,2503222401,0.086209,2,中島浩哉,4.0
2,2503222401,0.118014,3,柴田直哉,2.0
3,2503222401,0.089310,4,荒木颯斗,3.0
4,2503222401,0.040459,5,加木郁,5.0
...,...,...,...,...,...
1939,2503080112,0.143763,2,岡田憲行,2.0
1940,2503080112,0.096664,3,山口修路,4.0
1941,2503080112,0.111970,4,黄金井力,3.0
1942,2503080112,0.070160,5,桑島和宏,6.0


In [14]:
base_df['１着確率'] = df_result['1の確率'].values
base_df['予測順位'] = df_result['予測順位'].values

base_df['１着確率'] = (base_df['１着確率'] * 100).round(2)  # 小数点第3位で四捨五入
base_df['予測順位'] = base_df['予測順位'].astype(int)  # 整数に変換

class_mapping = {1: 'A1', 2: 'A2', 3: 'B1', 4: 'B2'}
base_df['級別'] = base_df['級別'].replace(class_mapping)

base_df

,艇番,選手登番,選手名,年齢,支部,体重,級別,全国勝率,全国2連率,当地勝率,当地2連率,モーターNO,モーター2連率,ボート2連率,開催地,レースID,１着確率,予測順位
0,1,5214,田中宏典,25,佐賀,53,B1,4.35,22.09,3.36,9.09,43,28.57,33.93,24,2503222401,36.15,1
1,2,4299,中島浩哉,42,長崎,56,B1,4.76,23.53,5.16,28.13,39,28.14,33.53,24,2503222401,8.62,4
2,3,4892,柴田直哉,34,福岡,53,B1,5.37,36.79,5.83,36.11,19,26.99,32.45,24,2503222401,11.80,2
3,4,5252,荒木颯斗,22,愛知,53,B1,4.65,26.21,4.35,23.08,66,36.90,29.82,24,2503222401,8.93,3
4,5,3581,加木郁,52,山口,52,B1,4.92,28.57,5.75,45.00,17,31.03,24.68,24,2503222401,4.05,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1003,2,3872,岡田憲行,48,大阪,53,A2,5.40,36.63,6.27,43.24,39,35.71,19.57,1,2503080112,14.38,2
1004,3,4367,山口修路,39,福岡,53,B1,4.88,25.58,4.20,13.33,52,63.83,43.14,1,2503080112,9.67,4
1005,4,4432,黄金井力,38,埼玉,57,A2,5.68,33.63,4.71,23.53,36,50.00,27.91,1,2503080112,11.20,3
1006,5,4379,桑島和宏,40,東京,56,A2,6.29,38.10,5.26,30.77,18,23.33,34.62,1,2503080112,7.02,6


In [15]:
print(base_df.columns.tolist())

['艇番', '選手登番', '選手名', '年齢', '支部', '体重', '級別', '全国勝率', '全国2連率', '当地勝率', '当地2連率', 'モーターNO', 'モーター2連率', 'ボート2連率', '開催地', 'レースID', '１着確率', '予測順位']


In [16]:
# レース場マッピング
racecourse_mapping = {
    '01': '桐生', '02': '戸田', '03': '江戸川', '04': '平和島', '05': '多摩川', '06': '浜名湖',
    '07': '蒲郡', '08': '常滑', '09': '津', '10': '三国', '11': 'びわこ', '12': '住之江',
    '13': '尼崎', '14': '鳴門', '15': '丸亀', '16': '児島', '17': '宮島', '18': '徳山',
    '19': '下関', '20': '若松', '21': '芦屋', '22': '福岡', '23': '唐津', '24': '大村'
}

def extract_race_info(race_id):
    date_str = str(race_id)[:6]
    year = 2000 + int(date_str[:2])
    month = int(date_str[2:4])
    day = int(date_str[4:6])
    date = pd.to_datetime(f"{year}-{month}-{day}")

    racecourse_code = str(race_id)[6:8]
    racecourse = racecourse_mapping.get(racecourse_code, "不明") # 存在しないコードの場合 "不明" を返す

    race_no = str(race_id)[8:10]
    
    return date, racecourse, race_no

# base_df に新しいカラムを追加
base_df[['日付', 'レース場', 'レースNo']] = base_df['レースID'].apply(lambda x: pd.Series(extract_race_info(x)))
base_df['日付'] = base_df['日付'].dt.date

In [17]:
#保存するフォルダのパスを指定します
folder_path = '予測結果'

# ファイルのパスをフォルダパスと結合してCSVファイルを保存します
file_path = os.path.join(folder_path, '予測結果（当日）.csv')
base_df.to_csv(file_path, index=False, encoding='shift-jis')

In [18]:
import sqlalchemy

# データベース接続情報
user = "boatrace_user"  # MySQLユーザー名
password = "your_password"  # MySQLパスワード
host = "db"  # MySQLホスト名 (localhostなど)
database = "boatrace_db"  # データベース名
table_name = "race_results" # テーブル名 (例: race_results)


# SQLAlchemyを使ってデータベースに接続
engine = sqlalchemy.create_engine(f"mysql+mysqlconnector://{user}:{password}@{host}/{database}")

# データフレームをデータベースに書き込み
try:
    base_df.to_sql(table_name, engine, if_exists='append', index=False)
    print(f"データフレームを '{table_name}' テーブルに書き込みました。")
except sqlalchemy.exc.OperationalError as e:  # 特定の例外をキャッチ
    if "1146" in str(e):  # テーブルが存在しないエラー (Error Code: 1146)
        print(f"テーブル '{table_name}' が存在しません。作成します。")
        base_df.to_sql(table_name, engine, if_exists='replace', index=False)  # テーブルを作成
        print(f"テーブル '{table_name}' を作成し、データを書き込みました。")
    else:
        print(f"その他のエラーが発生しました: {e}")
except Exception as e:
    print(f"予期せぬエラーが発生しました: {e}")

engine.dispose()

データフレームを 'race_results' テーブルに書き込みました。
